In [1]:
from pyspark.sql import functions as F

CLAIMS_OUT_PATH = "Files/source/claims_csv"

# Claim generation controls
CLAIM_RATE = 0.18   # ~18% of policies have at least one claim (adjust as needed)
MAX_CLAIMS_PER_POLICY = 3
SEED = 24


StatementMeta(, bb6deba8-4ebc-4643-9df0-20a2fa4a1b9f, 3, Finished, Available, Finished)

In [2]:
policies_path = "Files/source/policies_parquet"

policies = spark.read.parquet(policies_path).select(
    "policy_id", "customer_id", "agent_id",
    "policy_type", "plan_name",
    "start_date", "end_date", "policy_status",
    "sum_insured", "deductible"
).withColumn("start_date", F.to_date("start_date")) \
 .withColumn("end_date", F.to_date("end_date"))

display(policies)
print("Policies:", policies.count())

StatementMeta(, bb6deba8-4ebc-4643-9df0-20a2fa4a1b9f, 4, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 818e62a9-038a-4a93-887e-6f40dab9447f)

Policies: 120000


In [3]:
pol_with_claims = policies.filter(F.rand(SEED) < F.lit(CLAIM_RATE))
print("Policies with claims:", pol_with_claims.count())

StatementMeta(, bb6deba8-4ebc-4643-9df0-20a2fa4a1b9f, 5, Finished, Available, Finished)

Policies with claims: 21621


In [4]:
# For each selected policy, decide how many claims it has (1..MAX)
pol_claim_counts = (
    pol_with_claims
    .withColumn("claims_n", (F.floor(F.rand(SEED) * MAX_CLAIMS_PER_POLICY) + 1).cast("int"))
    .withColumn("claim_seq", F.expr("sequence(1, claims_n)"))
    .withColumn("claim_num", F.explode("claim_seq"))
    .drop("claim_seq")
)

# Random claim date between start and end (fallback if end < start)
pol_claim_counts = pol_claim_counts.withColumn(
    "safe_end_date",
    F.when(F.col("end_date").isNull() | (F.col("end_date") < F.col("start_date")),
           F.date_add(F.col("start_date"), 365))
     .otherwise(F.col("end_date"))
)

claims_base = pol_claim_counts.withColumn(
    "claim_date",
    F.to_date(
        F.expr("date_add(start_date, cast(rand() * greatest(datediff(safe_end_date, start_date), 1) as int))")
    )
)

display(claims_base.select("policy_id","claim_num","claim_date","policy_type","sum_insured"))

StatementMeta(, bb6deba8-4ebc-4643-9df0-20a2fa4a1b9f, 6, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, f8b442d6-94b9-4c2e-ae86-28f3e38b19b6)

In [5]:
# Claim reasons by type
auto_reasons   = ["Accident", "Theft", "Glass Damage", "Fire"]
health_reasons = ["Hospitalization", "Prescription", "Surgery", "Dental"]
home_reasons   = ["Water Damage", "Fire", "Storm", "Theft"]
life_reasons   = ["Critical Illness", "Disability", "Death Benefit", "Medical Emergency"]

# Pick reason list based on policy_type
reason_col = (
    F.when(F.col("policy_type") == "Auto",
           F.element_at(F.array([F.lit(x) for x in auto_reasons]), (F.floor(F.rand(SEED)*len(auto_reasons))+1).cast("int")))
     .when(F.col("policy_type") == "Health",
           F.element_at(F.array([F.lit(x) for x in health_reasons]), (F.floor(F.rand(SEED)*len(health_reasons))+1).cast("int")))
     .when(F.col("policy_type") == "Home",
           F.element_at(F.array([F.lit(x) for x in home_reasons]), (F.floor(F.rand(SEED)*len(home_reasons))+1).cast("int")))
     .otherwise(
           F.element_at(F.array([F.lit(x) for x in life_reasons]), (F.floor(F.rand(SEED)*len(life_reasons))+1).cast("int"))
     )
)

# Status distribution
claim_status = (
    F.when(F.rand(SEED) < 0.75, F.lit("Approved"))
     .when(F.rand(SEED) < 0.90, F.lit("Rejected"))
     .otherwise(F.lit("Pending"))
)

# Claim amount (<= sum_insured usually)
raw_claim_amt = F.round(F.col("sum_insured") * (F.rand(SEED) * 0.20 + 0.01), 2)  # 1% to 21% of sum insured

claims = (
    claims_base
    .withColumn("claim_id", F.concat(F.lit("CLM-"), F.sha2(F.concat_ws("|", F.col("policy_id"), F.col("claim_date").cast("string"), F.col("claim_num").cast("string")), 256)).substr(1, 18))
    .withColumn("claim_reason", reason_col)
    .withColumn("claim_status", claim_status)
    .withColumn("claim_amount", raw_claim_amt)
    # Approved amount: full amount if approved, else 0 (simple rule)
    .withColumn("approved_amount", F.when(F.col("claim_status") == "Approved", F.col("claim_amount")).otherwise(F.lit(0.0)))
    .withColumn("days_to_settle", F.when(F.col("claim_status") == "Approved", (F.floor(F.rand(SEED)*40)+3).cast("int")).otherwise(F.lit(None).cast("int")))
    .drop("claim_num", "claims_n", "safe_end_date")
)

display(claims.select("claim_id","policy_id","claim_date","claim_status","claim_amount","approved_amount","claim_reason"))
print("Claims rows:", claims.count())

StatementMeta(, bb6deba8-4ebc-4643-9df0-20a2fa4a1b9f, 7, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 91c9c80c-85fa-49f5-8ca7-83552611609c)

Claims rows: 43294


In [6]:
(claims
 .repartition(8)
 .write.mode("overwrite")
 .option("header", "true")
 .csv(CLAIMS_OUT_PATH))

print(f"✅ Claims CSV written to: {CLAIMS_OUT_PATH}")

StatementMeta(, bb6deba8-4ebc-4643-9df0-20a2fa4a1b9f, 8, Finished, Available, Finished)

✅ Claims CSV written to: Files/source/claims_csv


In [7]:
display(claims.groupBy("claim_status").count())
display(claims.groupBy("policy_type").count())

StatementMeta(, bb6deba8-4ebc-4643-9df0-20a2fa4a1b9f, 9, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 28068a3b-fdaa-4aa5-83a3-86e6dca392c8)

SynapseWidget(Synapse.DataFrame, 4f24f74a-44fa-4614-9aeb-486f01d2e6b8)